# Compressed Attribute

## Overview

Sometimes you want to compress the data before store to save DB space. For example, in an E-commerce data model, an order has many items like this: ``[{"item_name": "apple", "item_count": 12}, {"item_name": "banana", "item_count": 5}]``. There are lots of repeated information such as the keys ``"item_name"`` and ``"item_count"``.

**How it works**

Internally it always serializes your data into binary, and compress it, then send to DynamoDB.

## Define attribute to use Auto Compressed

In [1]:
import pynamodb_mate.api as pm
from rich import print as rprint

# Define the Data Model to use compressed attribute
class OrderModel(pm.Model):
    class Meta:
        table_name = f"pynamodb-mate-example-compress"
        region = "us-east-1"
        billing_mode = pm.constants.PAY_PER_REQUEST_BILLING_MODE

    order_id = pm.UnicodeAttribute(hash_key=True)

    # original value is unicode str
    description = pm.attributes.CompressedUnicodeAttribute(null=True)

    # original value is binary bytes
    image = pm.attributes.CompressedBinaryAttribute(null=True)

    # original value is any json serializable object
    items = pm.attributes.CompressedJSONDictAttribute(null=True)

OrderModel.create_table(wait=True)

## Write and Read the Item

In [2]:
# Create an item
order_id = "order_001"
description = "a fancy order!" * 1000
image = description.encode("utf-8") # a fake binary object
items = [
    {
        "item_id": "i_001",
        "item_name": "apple",
        "item_price": 2.4,
        "quantity": 8,
    },
    {
        "item_id": "i_002",
        "item_name": "banana",
        "item_price": 0.53,
        "quantity": 5,
    },
]
order = OrderModel(
    order_id=order_id,
    description=description,
    image=image,
    items=items,
)
# Save item to DynamoDB
order.save()
print(f"preview the DynamoDB item: {order.item_detail_console_url}")
print("you will see that the raw data in DynamoDB is compressed")

preview the DynamoDB item: https://us-east-1.console.aws.amazon.com/dynamodbv2/home?region=us-east-1#edit-item?table=pynamodb-mate-example-compress&itemMode=2&pk=order_001&sk&ref=%23item-explorer%3Ftable%3Dpynamodb-mate-example-compress&route=ROUTE_ITEM_EXPLORER
you will see that the raw data in DynamoDB is compressed


You can print the raw data in DynamoDB without using ``pynamodb_mate``. You will see the compressed data in binary format. The original ``image`` field value is around 15KB, however, it is just 112B in DynamoDB.  

In [3]:
import sys
from boto_session_manager import BotoSesManager

bsm = BotoSesManager()
res = bsm.dynamodb_client.get_item(TableName=OrderModel.Meta.table_name, Key={"order_id": {"S": "order_001"}})
rprint(res["Item"])
print("image value size = {}".format(sys.getsizeof(res["Item"]["image"]["B"])))

{
    'order_id': {'S': 'order_001'},
    'description': {
        'B': b'\x1f\x8b\x08\x00\x9e\x8fNf\x02\xff\xed\xc7\xb7\r\x00 
\x0c\x00\xb0W\xc2i\x11e\x04)\x1b\xdf\xf3\x03\xb3\xbd9c\xe5\xee7N\x8dY-\xcd\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\
xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xbe\xf6\x00\xf5\x991\x81\xb06\x00\x00'
    },
    'image': {
        'B': b'\x1f\x8b\x08\x00\x9e\x8fNf\x02\xff\xed\xc7\xb7\r\x00 
\x0c\x00\xb0W\xc2i\x11e\x04)\x1b\xdf\xf3\x03\xb3\xbd9c\xe5\xee7N\x8dY-\xcd\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\
xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xbe\xf6\x00\xf5\x991\x81\xb06\x00\x00'
    },
    'items': {
        'B': 
b'\x1f\x8b\x08\x00\x9e\x8fNf\x02\xff\x8b\xaeV\xca,I\xcd\x8d\xcfLQ\xb2RP\xca\x8c700T\xd2Q\x80\x88\xe5%\xe6\xa6\x82D\
x13\x0b\nrR\xe1\xa2\x05E\x99\xc9 
a#=\x13\xa0Paib^IfI%P\xc0\xa2VG\x01\xc34#\x0c\xd3\x92\x12\xf3\x80\x10\xc38\x03=ScT\xf3Lkc\x01I\x1e\xcf\x15\x9e\x00\
x00\x00'
    }
}

image value size = 112


When you get the value using ORM, you get the raw value.

In [4]:
# Get the value back and verify
order = OrderModel.get(order_id)
rprint(order.to_dict())

{
    'description': 'a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy o